### Model Training and  Hyperparameter Tuning

Since the purpose of the model to be trained is to predict future temperatures, the models to be used are regressions and not classifications.

There are 2 targets to be predicted which is the temperature at 2m and the apparent temperature, so the models that will be used supports multioutput regression. (More info in sklearn documentation: [Multiclass and multioutput algorithms](https://scikit-learn.org/stable/modules/multiclass.html))

Since this is also a time-dependent data, shuffling datasets is not recommended, and TimeSeriesSplit was used instead of KFold for cross validation to respect the temporal order of observations. 

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import TimeSeriesSplit, cross_validate, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor


In [16]:
df = pd.read_parquet('data/combined_hourly_data_mnl.parquet')

Split the datetime into year, month, day of the week, day of the year and hours

In [4]:
df['year'] = df.datetime.dt.year
df['month'] = df.datetime.dt.month
df['day_of_week'] = df.datetime.dt.day_of_week
df['day_of_year'] = df.datetime.dt.day_of_year
df['hour'] = df.datetime.dt.hour

del df['datetime']

Categorize the columns

In [5]:
numericals = df.columns.to_list()

In [6]:
targets = ['temperature_2m', 'apparent_temperature']
categories = ['weather_code', 'year'] 
cyclical_features = ['month', 'day_of_week', 'day_of_year', 'hour']

In [7]:
non_numeric_cols = targets + categories + cyclical_features

for cols in non_numeric_cols:
    numericals.remove(cols)

TimeSeriesSplit will be used instead of KFold because the dataset is time sensitive and shuffling the data in cross validation is not appropriate. 

I opted to not separate the train data into train and validation because the TimeSeriesSplit will split the dataset into a training and test set in every iteration.

Check the documentations for more info
- [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)
- [Visualizing cross-validation behavior in scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html)


In [8]:
total_len = len(df)
test_len= int(len(df) * 0.2)
train_len = total_len - test_len

In [9]:
total_len, train_len, test_len

(42168, 33735, 8433)

In [10]:
df_train = df.iloc[:train_len]
df_test = df.iloc[train_len:]

In [11]:
len(df_train), len(df_test)

(33735, 8433)

In [9]:
y = df[targets].values

y_train = y[:train_len]
y_test = y[train_len:]

In [10]:
tscv = TimeSeriesSplit(
    n_splits=4,
    gap=24,
    max_train_size=int(train_len * 0.8),
    test_size=int(train_len * 0.2),
)

Create transformations

In [11]:
def sin_transformer(max_val):
    return FunctionTransformer(lambda x: np.sin((x * 2 * np.pi) / max_val), feature_names_out='one-to-one')

def cos_transformer(max_val):
    return FunctionTransformer(lambda x: np.cos((x * 2 * np.pi) / max_val), feature_names_out='one-to-one')

In [12]:
transformations = [
    ('numerical', 'passthrough', numericals),
    ('day_of_year_sin', sin_transformer(365), ['day_of_year']),
    ('day_of_year_cos', cos_transformer(365), ['day_of_year']),
    ('month_sin', sin_transformer(12), ['month']),
    ('month_cos', cos_transformer(12), ['month']),
    ('day_of_week_sin', sin_transformer(7), ['day_of_week']),
    ('day_of_week_cos', cos_transformer(7), ['day_of_week']),
    ('hour_sin', sin_transformer(24), ['hour']),
    ('hour_cos', cos_transformer(24), ['hour']),
    ('category', OneHotEncoder(dtype='int32', handle_unknown='ignore'), categories)
]

transformer = ColumnTransformer(
    transformations,
    remainder='drop'
)

In [13]:
def evaluate(model, X, y, cv, model_prop=None, model_step=None):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
        return_estimator=model_prop is not None,
    )
    if model_prop is not None:
        if model_step is not None:
            values = [
                getattr(m[model_step], model_prop) for m in cv_results["estimator"]
            ]
        else:
            values = [getattr(m, model_prop) for m in cv_results["estimator"]]
        print(f"Mean model.{model_prop} = {np.mean(values)}")
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )

#### Linear Regression Model

In [14]:
lf_pipeline = Pipeline([
    ('transformer', transformer),
    ('lr', LinearRegression())
])

In [15]:
evaluate(lf_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.168 +/- 0.033
Root Mean Squared Error: 0.217 +/- 0.039


#### Ridge Model

In [14]:
params = {
    'rd__alpha': [0, 0.01, 0.1, 1, 10, 100]
}

rd_pipeline = Pipeline([
    ('transformer', transformer),
    ('rd', Ridge(random_state=42, max_iter=1000))
])

model_rd = RandomizedSearchCV(
    estimator=rd_pipeline,
    param_distributions=params,
    cv=tscv,
    random_state=42
)

model_rd.fit(df_train, y_train)

c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                                              ('hour_cos',
                                                                               FunctionTransformer(feature_names_out='one-to-one',
                                                                                                   func=<function cos_transformer.<locals>.<lambda> at 0x0000016F4EE46D40>),
                                                                               ['hour']),
                                                                              ('category',
                                                                               OneHotEncoder(dtype='int32',
                                                                                             handle_unknown='ignore'),
                                                                               ['weather_code',
                                                                                'year'])])),
                                             ('rd',
                                              Ridge(max_iter=1000,
                                                    random_state=42))]),
                   param_distributions={'rd__alpha': [0, 0.01, 0.1, 1, 10,
                                                      100]},
                   random_state=42)

In [15]:
print(f'best score: {model_rd.best_score_}')
print(f'best parameters: {model_rd.best_params_}')

best score: 0.9939827935279386
best parameters: {'rd__alpha': 10}


cross validate the best parameter of the Ridge model

In [16]:
rd_pipeline = Pipeline([
    ('transformer', transformer),
    ('rd', Ridge(alpha=10, random_state=42, max_iter=1000))
])

evaluate(rd_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.155 +/- 0.018
Root Mean Squared Error: 0.203 +/- 0.023


Ridge model is a little bit better than using Linear Regression model

#### Lasso

In [31]:
params = {
    'ls__alpha': [0, 0.01, 0.1, 1, 10, 100]
}

ls_pipeline = Pipeline([
    ('transformer', transformer),
    ('ls', Lasso(
        max_iter=1000,
        random_state=42,
        selection='random'
    ))
])

model_ls = RandomizedSearchCV(
    estimator=ls_pipeline,
    param_distributions=params,
    cv=tscv,
    random_state=42
)

model_ls.fit(df_train, y_train)

c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not con

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                                              ('hour_cos',
                                                                               FunctionTransformer(feature_names_out='one-to-one',
                                                                                                   func=<function cos_transformer.<locals>.<lambda> at 0x00000215D4B080E0>),
                                                                               ['hour']),
                                                                              ('category',
                                                                               OneHotEncoder(dtype='int32',
                                                                                             handle_unknown='ignore'),
                                                                               ['weather_code',
                                                                                'year'])])),
                                             ('ls',
                                              Lasso(random_state=42,
                                                    selection='random'))]),
                   param_distributions={'ls__alpha': [0, 0.01, 0.1, 1, 10,
                                                      100]},
                   random_state=42)

In [32]:
print(f'best score: {model_ls.best_score_}')
print(f'best parameters: {model_ls.best_params_}')

best score: 0.9931669742699077
best parameters: {'ls__alpha': 0.01}


In [33]:
ls_pipeline = Pipeline([
    ('transformer', transformer),
    ('ls', Lasso(
        alpha=0.01,
        max_iter=1000,
        random_state=42,
        selection='random'
    ))
])

evaluate(ls_pipeline, df_train, y_train, cv=tscv)

c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.017e+02, tolerance: 4.918e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.164e+02, tolerance: 6.855e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number o

Mean Absolute Error:     0.167 +/- 0.015
Root Mean Squared Error: 0.217 +/- 0.016


c:\Users\User\.virtualenvs\ml-zoomcamp-temp-prediction-UD_BFAAe\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.821e+03, tolerance: 2.302e+01
  model = cd_fast.enet_coordinate_descent(


Ridge Model is still better than the Lasso

#### Random Forest Regression Model

In [19]:
params = {
    'rf__n_estimators': np.arange(10, 210, 10),
    'rf__max_depth': [None, 5, 10, 15, 20],
    'rf__min_samples_leaf': [1, 3, 5, 10, 50],
}

rf_pipeline = Pipeline([
    ('transformer', transformer),
    ('rf', RandomForestRegressor(n_jobs=-1, random_state=42))
])

model_rf = RandomizedSearchCV(
    estimator=rf_pipeline,
    param_distributions=params,
    cv=tscv,
    random_state=42
)

model_rf.fit(df_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                                              ('category',
                                                                               OneHotEncoder(dtype='int32',
                                                                                             handle_unknown='ignore'),
                                                                               ['weather_code',
                                                                                'year'])])),
                                             ('rf',
                                              RandomForestRegressor(n_jobs=-1,
                                                                    random_state=42))]),
                   param_distributions={'rf__max_depth': [None, 5, 10, 15, 20],
                                        'rf__min_samples_leaf': [1, 3, 5, 10,
                                                                 50],
                                        'rf__n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
                   random_state=42)

In [20]:
print(f'best score: {model_rf.best_score_}')
print(f'best parameters: {model_rf.best_params_}')

best score: 0.9739195908648586
best parameters: {'rf__n_estimators': np.int64(110), 'rf__min_samples_leaf': 5, 'rf__max_depth': 20}


In [22]:
rf_pipeline = Pipeline([
    ('transformer', transformer),
    ('rf', RandomForestRegressor(
        n_estimators=110,
        max_depth=5,
        min_samples_leaf=20,
        n_jobs=-1,
        random_state=42
    ))
])

evaluate(rf_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.696 +/- 0.053
Root Mean Squared Error: 0.904 +/- 0.072


Random Forest has significanly worse MAE and RMSE than Ridge Model

#### Histogram Gradient Boosting Trees

In [27]:
params = {
    'hgbr__estimator__learning_rate': [1.0, 0.3, 0.1, 0.05, 0.01],
    'hgbr__estimator__max_iter': np.arange(10, 210, 10),
    'hgbr__estimator__max_depth': [None, 5, 10, 15, 20],
    'hgbr__estimator__min_samples_leaf': np.arange(5, 150, 5),
}

hgbr_pipeline = Pipeline([
    ('transformer', transformer),
    ('hgbr', MultiOutputRegressor(
        HistGradientBoostingRegressor(
            random_state=42
        ))
    )
])

model_hgbr = RandomizedSearchCV(
    estimator=hgbr_pipeline,
    param_distributions=params,
    cv=tscv,
    random_state=42
)

model_hgbr.fit(df_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                   param_distributions={'hgbr__estimator__learning_rate': [1.0,
                                                                           0.3,
                                                                           0.1,
                                                                           0.05,
                                                                           0.01],
                                        'hgbr__estimator__max_depth': [None, 5,
                                                                       10, 15,
                                                                       20],
                                        'hgbr__estimator__max_iter': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200]),
                                        'hgbr__estimator__min_samples_leaf': array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130,
       135, 140, 145])},
                   random_state=42)

In [28]:
print(f'best score: {model_hgbr.best_score_}')
print(f'best parameters: {model_hgbr.best_params_}')

best score: 0.9844548597212543
best parameters: {'hgbr__estimator__min_samples_leaf': np.int64(10), 'hgbr__estimator__max_iter': np.int64(170), 'hgbr__estimator__max_depth': None, 'hgbr__estimator__learning_rate': 0.1}


In [29]:
hgbr_pipeline = Pipeline([
    ('transformer', transformer),
    ('hgbr', MultiOutputRegressor(
        HistGradientBoostingRegressor(
            learning_rate=0.1,
            max_iter=170,
            max_depth=None,
            min_samples_leaf=10,
            random_state=42
        ))
    )
])

evaluate(hgbr_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.213 +/- 0.092
Root Mean Squared Error: 0.304 +/- 0.126


#### Gradient Boosting (XGBoost)

Using XGBRegressor `multi_strategy = multi_output_tree`

In [23]:
hyperparameter_params = {
    'xgb__n_estimators': np.arange(10, 210, 10),
    'xgb__learning_rate': [1.0, 0.3, 0.1, 0.05, 0.01],
    'xgb__max_depth': [3, 4, 6, 8, 10, 12],
    'xgb__min_child_weight': [1, 10, 20, 30],
}

xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', XGBRegressor(
        tree_method='hist',
        multi_strategy='multi_output_tree',
        objective='reg:squarederror',
        random_state=42, 
        n_jobs=-1, 
        verbosity=0,
    ))
])

model_xgb = RandomizedSearchCV(
    estimator=xgb_pipeline,
    param_distributions=hyperparameter_params,
    cv=tscv,
    random_state=42
)

model_xgb.fit(df_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                           multi_strategy='multi_output_tree',
                                                           n_estimators=None,
                                                           n_jobs=-1,
                                                           num_parallel_tree=None,
                                                           random_state=42, ...))]),
                   param_distributions={'xgb__learning_rate': [1.0, 0.3, 0.1,
                                                               0.05, 0.01],
                                        'xgb__max_depth': [3, 4, 6, 8, 10, 12],
                                        'xgb__min_child_weight': [1, 10, 20,
                                                                  30],
                                        'xgb__n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
                   random_state=42)

In [24]:
print(f'best score: {model_xgb.best_score_}')
print(f'best parameters: {model_xgb.best_params_}')

best score: 0.9862629992063611
best parameters: {'xgb__n_estimators': np.int64(190), 'xgb__min_child_weight': 10, 'xgb__max_depth': 8, 'xgb__learning_rate': 0.1}


In [25]:
xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', XGBRegressor(
        n_estimators=190,
        learning_rate=0.1,
        max_depth=8,
        min_child_weight=10,
        tree_method='hist',
        multi_strategy='multi_output_tree',
        objective='reg:squarederror',
        random_state=42, 
        n_jobs=-1, 
        verbosity=0,
    ))
])

evaluate(xgb_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.193 +/- 0.075
Root Mean Squared Error: 0.292 +/- 0.107


using `MultiOutputRegressor` with XGBRegressor `multi_strategy = multi_output_tree`

In [26]:
hyperparameter_params = {
    'xgb__estimator__n_estimators': np.arange(10, 210, 10),
    'xgb__estimator__learning_rate': [1.0, 0.3, 0.1, 0.05, 0.01],
    'xgb__estimator__max_depth': [3, 4, 6, 8, 10, 12],
    'xgb__estimator__min_child_weight': [1, 10, 20, 30],
}

xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', MultiOutputRegressor(
        XGBRegressor(
            tree_method='hist',
            multi_strategy='multi_output_tree',
            objective='reg:squarederror',
            random_state=42, 
            n_jobs=-1, 
            verbosity=0,
        ))
    )
])

model_xgb_mor_mot = RandomizedSearchCV(
    estimator=xgb_pipeline,
    param_distributions=hyperparameter_params,
    cv=tscv,
    random_state=42
)

model_xgb_mor_mot.fit(df_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                                                          n_jobs=-1,
                                                                                          num_parallel_tree=None,
                                                                                          random_state=42, ...)))]),
                   param_distributions={'xgb__estimator__learning_rate': [1.0,
                                                                          0.3,
                                                                          0.1,
                                                                          0.05,
                                                                          0.01],
                                        'xgb__estimator__max_depth': [3, 4, 6,
                                                                      8, 10,
                                                                      12],
                                        'xgb__estimator__min_child_weight': [1,
                                                                             10,
                                                                             20,
                                                                             30],
                                        'xgb__estimator__n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
                   random_state=42)

In [27]:
print(f'best score: {model_xgb_mor_mot.best_score_}')
print(f'best parameters: {model_xgb_mor_mot.best_params_}')

best score: 0.9817097262175026
best parameters: {'xgb__estimator__n_estimators': np.int64(190), 'xgb__estimator__min_child_weight': 10, 'xgb__estimator__max_depth': 6, 'xgb__estimator__learning_rate': 0.05}


In [28]:
xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', MultiOutputRegressor(
        XGBRegressor(
            n_estimators=190,
            learning_rate=0.05,
            max_depth=6,
            min_child_weight=10,
            tree_method='hist',
            multi_strategy='multi_output_tree',
            objective='reg:squarederror',
            random_state=42, 
            n_jobs=-1, 
            verbosity=0,
        )
    ))
])

evaluate(xgb_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.230 +/- 0.092
Root Mean Squared Error: 0.332 +/- 0.127


using `MultiOutputRegressor` only (No XGBRegressor `multi_strategy = multi_output_tree` attribute)

XGBRegressor `multi_strategy = multi_output_tree` attribute is currently an experimental feature, so I also want to test without using it

In [61]:
hyperparameter_params = {
    'xgb__estimator__n_estimators': np.arange(10, 210, 10),
    'xgb__estimator__learning_rate': [1.0, 0.3, 0.1, 0.05, 0.01],
    'xgb__estimator__max_depth': [3, 4, 6, 8, 10, 12],
    'xgb__estimator__min_child_weight': [1, 10, 20, 30],
}

xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', MultiOutputRegressor(
        XGBRegressor(
            tree_method='hist',
            objective='reg:squarederror',
            random_state=42, 
            n_jobs=-1, 
            verbosity=0,
        ))
    )
])

model_xgb_mor = RandomizedSearchCV(
    estimator=xgb_pipeline,
    param_distributions=hyperparameter_params,
    cv=tscv,
    random_state=42,
    scoring=('neg_mean_squared_error'),
)

model_xgb_mor.fit(df_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=24, max_train_size=26988, n_splits=4, test_size=6747),
                   estimator=Pipeline(steps=[('transformer',
                                              ColumnTransformer(transformers=[('numerical',
                                                                               'passthrough',
                                                                               ['relative_humidity_2m',
                                                                                'dew_point_2m',
                                                                                'precipitation',
                                                                                'rain',
                                                                                'pressure_msl',
                                                                                'surface_pressure',
                                                                                'cloud_cover',
                                                                                'cloud_cover_low',
                                                                                'cloud_cover_mid',
                                                                                'cloud_cove...
                                                                                          random_state=42, ...)))]),
                   param_distributions={'xgb__estimator__learning_rate': [1.0,
                                                                          0.3,
                                                                          0.1,
                                                                          0.05,
                                                                          0.01],
                                        'xgb__estimator__max_depth': [3, 4, 6,
                                                                      8, 10,
                                                                      12],
                                        'xgb__estimator__min_child_weight': [1,
                                                                             10,
                                                                             20,
                                                                             30],
                                        'xgb__estimator__n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
                   random_state=42, scoring='neg_mean_squared_error')

In [62]:
print(f'best score: {model_xgb_mor.best_score_}')
print(f'best parameters: {model_xgb_mor.best_params_}')

best score: -0.13997847361141297
best parameters: {'xgb__estimator__n_estimators': np.int64(190), 'xgb__estimator__min_child_weight': 10, 'xgb__estimator__max_depth': 6, 'xgb__estimator__learning_rate': 0.05}


In [31]:
xgb_pipeline = Pipeline([
    ('transformer', transformer),
    ('xgb', MultiOutputRegressor(
        XGBRegressor(
            n_estimators=190,
            learning_rate=0.05,
            max_depth=6,
            min_child_weight=10,
            tree_method='hist',
            objective='reg:squarederror',
            random_state=42, 
            n_jobs=-1, 
            verbosity=0,
        )
    ))
])

evaluate(xgb_pipeline, df_train, y_train, cv=tscv)

Mean Absolute Error:     0.230 +/- 0.092
Root Mean Squared Error: 0.332 +/- 0.127


Ridge model is still better than the XGBRegressor

The final model to be used is the Ridge Model

To avoid additional work of transforming lambda functions to named function, `cloudpickle` will be used instead of `pickle`

In [11]:
import cloudpickle

In [12]:
tscv = TimeSeriesSplit(
    n_splits=4,
    gap=24,
    max_train_size=int(train_len * 0.8),
    test_size=int(train_len * 0.2),
)

Create transformations

In [13]:
def sin_transformer(max_val):
    return FunctionTransformer(lambda x: np.sin((x * 2 * np.pi) / max_val), feature_names_out='one-to-one')

def cos_transformer(max_val):
    return FunctionTransformer(lambda x: np.cos((x * 2 * np.pi) / max_val), feature_names_out='one-to-one')

In [14]:
transformations = [
    ('numerical', 'passthrough', numericals),
    ('day_of_year_sin', sin_transformer(365), ['day_of_year']),
    ('day_of_year_cos', cos_transformer(365), ['day_of_year']),
    ('month_sin', sin_transformer(12), ['month']),
    ('month_cos', cos_transformer(12), ['month']),
    ('day_of_week_sin', sin_transformer(7), ['day_of_week']),
    ('day_of_week_cos', cos_transformer(7), ['day_of_week']),
    ('hour_sin', sin_transformer(24), ['hour']),
    ('hour_cos', cos_transformer(24), ['hour']),
    ('category', OneHotEncoder(dtype='int32', handle_unknown='ignore'), categories)
]

transformer = ColumnTransformer(
    transformations,
    remainder='drop'
)

In [16]:
rd_pipeline = Pipeline([
    ('transformer', transformer),
    ('rd', Ridge(alpha=10, random_state=42, max_iter=1000))
])

In [17]:
rd_pipeline.fit(df_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('numerical', 'passthrough',
                                                  ['relative_humidity_2m',
                                                   'dew_point_2m',
                                                   'precipitation', 'rain',
                                                   'pressure_msl',
                                                   'surface_pressure',
                                                   'cloud_cover',
                                                   'cloud_cover_low',
                                                   'cloud_cover_mid',
                                                   'cloud_cover_high',
                                                   'et0_fao_evapotranspiration',
                                                   'vapour_pressure_deficit',
                                                   'wind_speed_10m',
                                                   'wind_speed_100m',
                                                   'wind...
                                                                      func=<function sin_transformer.<locals>.<lambda> at 0x0000021877DE20C0>),
                                                  ['hour']),
                                                 ('hour_cos',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function cos_transformer.<locals>.<lambda> at 0x0000021877DE2160>),
                                                  ['hour']),
                                                 ('category',
                                                  OneHotEncoder(dtype='int32',
                                                                handle_unknown='ignore'),
                                                  ['weather_code', 'year'])])),
                ('rd', Ridge(alpha=10, max_iter=1000, random_state=42))])

In [18]:
with open('model.bin', 'wb') as f_out:
    cloudpickle.dump(rd_pipeline, f_out)

test the pickle file

In [1]:
import cloudpickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    model = cloudpickle.load(f_in)

In [3]:
input = {
    'datetime': ['2024-10-22 20:00:00'],
    'relative_humidity_2m': [95.0],
    'dew_point_2m': [24.3],
    'precipitation': [1.0],
    'rain': [1.0],
    'weather_code': [55.0],
    'pressure_msl': [1004.0],
    'surface_pressure': [1003.0],
    'cloud_cover': [87.0],
    'cloud_cover_low': [0.0],
    'cloud_cover_mid': [95.0],
    'cloud_cover_high': [100.0],
    'et0_fao_evapotranspiration': [0.0],
    'vapour_pressure_deficit': [0.16],
    'wind_speed_10m': [8.3],
    'wind_speed_100m': [8.0],
    'wind_direction_10m': [90.0],
    'wind_direction_100m': [98.0],
    'wind_gusts_10m': [18.4],
    'soil_temperature_0_to_7cm': [26.9],
    'soil_temperature_7_to_28cm': [28.7],
    'soil_temperature_28_to_100cm': [28.9],
    'soil_temperature_100_to_255cm': [29.2],
    'soil_moisture_0_to_7cm': [0.46],
    'soil_moisture_7_to_28cm': [0.426],
    'soil_moisture_28_to_100cm': [0.423],
    'soil_moisture_100_to_255cm': [0.493],
    'shortwave_radiation': [0.0],
    'direct_radiation': [0.0],
    'diffuse_radiation': [0.0],
    'direct_normal_irradiance': [0.0],
    'global_tilted_irradiance': [0.0],
    'terrestrial_radiation': [0.0],
    'shortwave_radiation_instant': [0.0],
    'direct_radiation_instant': [0.0],
    'diffuse_radiation_instant': [0.0],
    'direct_normal_irradiance_instant': [0.0],
    'global_tilted_irradiance_instant': [0.0],
    'terrestrial_radiation_instant': [0.0]
}

In [6]:
x = pd.DataFrame.from_dict(input)
x.datetime = pd.to_datetime(x.datetime)
x['year'] = x.datetime.dt.year
x['month'] = x.datetime.dt.month
x['day_of_week'] = x.datetime.dt.day_of_week
x['day_of_year'] = x.datetime.dt.day_of_year
x['hour'] = x.datetime.dt.hour

In [7]:
model.predict(x)

array([[25.12338482, 30.25528301]])